## TRABALHO DE CONCLUSÃO DE CURSO EM CIÊNCIA DE DADOS - PUC MINAS
## Aluno: Anderson Lhoret
## MODELO PREDITIVO DE DURAÇÃO DE OPERAÇÃO DE TRÂNSITO ADUANEIRO
# Notebook 1.1: Coleta de dados - Google Maps API Distance Matrix

In [1]:
import json
import pandas as pd
import requests

In [2]:
"""
Recebe a string json retornada pela API Distance Matrix do Google e recupera
os valores de distancia e duração. Salva os elementos 'text' e 'value'.
"""
def parse_distance_matrix_response(resposta):
    if resposta is None:
        return None
    
    origin_addresses = None
    destination_addresses = None
    distance_text = None
    distance_value = None
    duration_text = None
    duration_value = None
    
    # The variable data contains the JSON as a Python dictionary
    data = json.loads(resposta)    
    if data['status'] == 'OK':        
        origin_addresses = data['origin_addresses'][0]
        destination_addresses = data['destination_addresses'][0]
        row = data['rows'][0]
        elemento = row['elements'][0]        
        if elemento['status'] == 'OK':
            distance_text = elemento['distance']['text']
            distance_value = elemento['distance']['value']
            duration_text = elemento['duration']['text']
            duration_value = elemento['duration']['value']
    return (origin_addresses, destination_addresses, distance_text, distance_value, duration_text, duration_value)

In [3]:
def isNaN(num):
    return num != num

In [4]:
# Arquivo com as coordenadas (latitude e longitude) dos pares origens & destino das operações de trânsito aduaneiro.
input_file = 'tabela_coordenadas.csv'

# Arquivo salvo com as distâncias entre as coordenadas dos locais de origem e destino.
output_file = 'tabela_distancias_ok.csv'

# Chave para acessar serviço da plataforma Google Maps
API_KEY = ''

In [5]:
df = pd.read_csv(input_file)

In [6]:
# Acrescenta as novas colunas para distância e duração
df['distancia_text'] = ''
df['distancia_value'] = ''
df['duracao_text_Google'] = ''
df['duracao_value_Google'] = ''
df['endereco_origem'] = ''
df['endereco_destino'] = ''

In [ ]:
for i in range(len(df)):
    print ('Loop: {}'.format(i))

    # latitude e longitude da origem
    origem = df.iloc[i,0].astype(str) + '%2C' + df.iloc[i,1].astype(str)
    
    # latitude e longitude do destino
    destino = df.iloc[i,2].astype(str) + '%2C' + df.iloc[i,3].astype(str)

    # URL da API DistanceMatrix do Google
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?origins='+ origem +'&destinations='+ destino +'&key='  + API_KEY

    #print(url)
    payload={}
    headers = {}

    response = requests.request("GET", url, headers=headers, data=payload)
    #print(response.text)
    endereco_origem, endereco_destino, distance_text, distance_value, duration_text, duration_value = parse_distance_matrix_response(response.text)
    df.iloc[i, 4] = distance_text
    df.iloc[i, 5] = distance_value
    df.iloc[i, 6] = duration_text
    df.iloc[i, 7] = duration_value
    df.iloc[i, 8] = endereco_origem
    df.iloc[i, 9] = endereco_destino

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   origem_latitude           1275 non-null   float64
 1   origem_longitude          1275 non-null   float64
 2   dest_realizado_latitude   1275 non-null   float64
 3   dest_realizado_longitude  1275 non-null   float64
 4   distancia_text            1233 non-null   object 
 5   distancia_value           1233 non-null   object 
 6   duracao_text_Google       1233 non-null   object 
 7   duracao_value_Google      1233 non-null   object 
 8   endereco_origem           1274 non-null   object 
 9   endereco_destino          1274 non-null   object 
dtypes: float64(4), object(6)
memory usage: 99.7+ KB


In [11]:
# Exclui linhas com colunas vazias que sao carregadas como NaN no DataFrame
df = df[~isNaN(df['distancia_text'])]

# Exclui linhas cuja distancia nao está em km
df = df[(df['distancia_text'].str.contains('km'))]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1126 entries, 0 to 1274
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   origem_latitude           1126 non-null   float64
 1   origem_longitude          1126 non-null   float64
 2   dest_realizado_latitude   1126 non-null   float64
 3   dest_realizado_longitude  1126 non-null   float64
 4   distancia_text            1126 non-null   object 
 5   distancia_value           1126 non-null   object 
 6   duracao_text_Google       1126 non-null   object 
 7   duracao_value_Google      1126 non-null   object 
 8   endereco_origem           1126 non-null   object 
 9   endereco_destino          1126 non-null   object 
dtypes: float64(4), object(6)
memory usage: 96.8+ KB


In [14]:
# Salva novo arquivo CSV com as informações obtidas da API Google
df.to_csv(output_file, index = False)